In [2]:
# 1. Imports
import os
import numpy as np
import pandas as pd
import json
import time

# Set a seed for reproducibility
np.random.seed(42)


In [3]:


# [cite_start]2. Data Loading and Preprocessing (Reusable Code) [cite: 48]
def load_and_preprocess_data(file_path="Housing.csv"):
    """Loads and prepares the housing data."""
    df = pd.read_csv(file_path)

    # --- Preprocessing Steps ---
    for col in ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea"]:
        df[col] = df[col].map({"yes": 1, "no": 0})
    df = pd.get_dummies(df, columns=["furnishingstatus"], prefix="furnish", drop_first=True)
    df.fillna(df.mean(numeric_only=True), inplace=True)

    target_scaler = np.mean(df["price"]) / 10
    df["price"] = df["price"] / target_scaler

    df["area_bedroom_ratio"] = df["area"] / (df["bedrooms"] + 1)

    # --- Feature and Target Split ---
    X = df.drop(columns=["price"]).values.astype(float)
    y = y = df["price"].values.reshape(-1, 1).astype(float)

    # --- Normalization ---
    X_mean = X.mean(axis=0)
    X_std = X.std(axis=0)
    X_std[X_std == 0] = 1
    X = (X - X_mean) / X_std

    # --- Train/Test Split ---
    n = X.shape[0]
    idx = np.arange(n)
    np.random.shuffle(idx)
    split = int(n * 0.8)
    train_idx, test_idx = idx[:split], idx[split:]

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    return X_train, X_test, y_train, y_test, target_scaler

# Activation and loss functions (can be placed here or inside the main function)
def relu(z):
    return np.maximum(0, z)

def drelu(z):
    return (z > 0).astype(float)

# --- Updated Metric Functions ---
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)



In [4]:

# [cite_start]3. Define Trainable Model Function [cite: 49]
def train_model(X_train, y_train, X_test, y_test, learning_rate, n_hidden, n_epochs, target_scaler):
    """
    A function that takes hyperparameters, trains a model, and returns performance.
    """
    input_size = X_train.shape[1]
    output_size = 1

    # Initialize weights
    W1 = np.random.randn(input_size, n_hidden) * 0.01
    b1 = np.zeros((1, n_hidden))
    W2 = np.random.randn(n_hidden, output_size) * 0.01
    b2 = np.zeros((1, output_size))

    # Training loop
    for ep in range(n_epochs):
        # Forward pass
        z1 = X_train.dot(W1) + b1
        a1 = relu(z1)
        z2 = a1.dot(W2) + b2
        y_pred = z2

        # Backpropagation
        dz2 = (y_pred - y_train) * (2 / y_train.shape[0])
        dW2 = a1.T.dot(dz2)
        db2 = np.sum(dz2, axis=0, keepdims=True)
        da1 = dz2.dot(W2.T)
        dz1 = da1 * drelu(z1)
        dW1 = X_train.T.dot(dz1)
        db1 = np.sum(dz1, axis=0, keepdims=True)

        # Update parameters
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2

    # Evaluation on the test set
    test_preds_scaled = relu(X_test.dot(W1) + b1).dot(W2) + b2
    
    # Rescale to original price units for metrics
    test_preds_actual = test_preds_scaled * target_scaler
    y_test_actual = y_test * target_scaler

    # [cite_start]--- Calculate all regression metrics --- [cite: 38]
    metrics = {
        "mse": mse(y_test_actual, test_preds_actual),
        "rmse": rmse(y_test_actual, test_preds_actual),
        "mae": mae(y_test_actual, test_preds_actual),
        "r2": r2_score(y_test_actual, test_preds_actual)
    }
    
    model_weights = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    
    return metrics, model_weights


# [cite_start]4. Hyperparameter Tuning (Manual) [cite: 51]
print("Starting Hyperparameter Tuning...")

# Load data once
X_train, X_test, y_train, y_test, target_scaler = load_and_preprocess_data()

# [cite_start]Define hyperparameter search space [cite: 52, 53, 54, 56]
learning_rates = [0.001, 0.005]
hidden_units = [16, 32]
epochs_list = [500, 1000]

results = []
best_rmse = float('inf')
best_hyperparams = {}
best_model_weights = {}

# [cite_start]--- Nested loops to test combinations --- [cite: 58]
run_number = 1
for lr in learning_rates:
    for hidden in hidden_units:
        for epochs in epochs_list:
            start_time = time.time()
            print(f"\n--- Running combination {run_number}: LR={lr}, Hidden Units={hidden}, Epochs={epochs} ---")
            
            # Train the model with the current set of hyperparameters
            current_metrics, current_weights = train_model(
                X_train, y_train, X_test, y_test, 
                learning_rate=lr, n_hidden=hidden, n_epochs=epochs, 
                target_scaler=target_scaler
            )
            
            end_time = time.time()
            # --- Updated print statement to show multiple metrics ---
            print(f"Result: Test RMSE = {current_metrics['rmse']:,.2f} | Test R² = {current_metrics['r2']:.4f} (took {end_time - start_time:.2f}s)")
            
            # [cite_start]Track performance for each combination [cite: 58]
            run_info = {
                'learning_rate': lr,
                'hidden_units': hidden,
                'epochs': epochs,
                'metrics': current_metrics
            }
            results.append(run_info)

            # Check if this is the best model so far based on RMSE
            if current_metrics['rmse'] < best_rmse:
                print(f"✨ New best model found! Previous best RMSE: {best_rmse:,.2f}")
                best_rmse = current_metrics['rmse']
                best_hyperparams = run_info
                best_model_weights = current_weights
            
            run_number += 1

# [cite_start]5. Best Model [cite: 59]
print("\n" + "="*40)
print("Hyperparameter Tuning Complete!")
print("="*40)
# --- Updated to display all metrics for the best model ---
print("\nBest Hyperparameters Found:")
best_info_to_print = {k: v for k, v in best_hyperparams.items() if k != 'metrics'}
print(json.dumps(best_info_to_print, indent=4))
print("\nPerformance of Best Model:")
print(json.dumps(best_hyperparams.get('metrics', {}), indent=4))


# [cite_start]6. Save Best Model & Results [cite: 62]
# Create a directory for the best model's artifacts
os.makedirs("best_model", exist_ok=True)

# [cite_start]Save the weights of the best model [cite: 63]
for name, weights in best_model_weights.items():
    np.save(f"best_model/{name}.npy", weights)
print("\nBest model weights saved in 'best_model/' directory.")

# [cite_start]Save the best hyperparameters and results to a JSON file [cite: 63]
with open("best_model/best_model_details.json", "w") as f:
    # Convert numpy float types to standard python floats for JSON serialization
    serializable_hyperparams = best_hyperparams.copy()
    serializable_hyperparams['metrics'] = {k: float(v) for k, v in serializable_hyperparams['metrics'].items()}
    json.dump(serializable_hyperparams, f, indent=4)
print("Best model details saved to 'best_model/best_model_details.json'.")


# [cite_start]7. Conclusion [cite: 65]
print("\n--- Conclusion ---")
print("The hyperparameter tuning process systematically evaluated multiple combinations of learning rates, hidden units, and epochs.")
print("The baseline model (from Notebook 1) might have had a decent performance, but by exploring different configurations, we found a model with a lower Test RMSE, indicating better predictive accuracy.")
print(f"The best model, with LR={best_hyperparams['learning_rate']}, Hidden Units={best_hyperparams['hidden_units']}, and {best_hyperparams['epochs']} epochs, provides a more optimized solution for this dataset.")
print("This demonstrates the critical importance of tuning, as default or initial guess hyperparameters are rarely optimal.")

Starting Hyperparameter Tuning...

--- Running combination 1: LR=0.001, Hidden Units=16, Epochs=500 ---
Result: Test RMSE = 1,172,992.71 | Test R² = 0.6082 (took 0.04s)
✨ New best model found! Previous best RMSE: inf

--- Running combination 2: LR=0.001, Hidden Units=16, Epochs=1000 ---
Result: Test RMSE = 1,095,513.05 | Test R² = 0.6583 (took 0.08s)
✨ New best model found! Previous best RMSE: 1,172,992.71

--- Running combination 3: LR=0.001, Hidden Units=32, Epochs=500 ---
Result: Test RMSE = 1,148,619.49 | Test R² = 0.6243 (took 0.05s)

--- Running combination 4: LR=0.001, Hidden Units=32, Epochs=1000 ---
Result: Test RMSE = 1,099,796.04 | Test R² = 0.6556 (took 0.12s)

--- Running combination 5: LR=0.005, Hidden Units=16, Epochs=500 ---
Result: Test RMSE = 1,081,940.61 | Test R² = 0.6667 (took 0.04s)
✨ New best model found! Previous best RMSE: 1,095,513.05

--- Running combination 6: LR=0.005, Hidden Units=16, Epochs=1000 ---
Result: Test RMSE = 1,073,062.41 | Test R² = 0.6721 (too